# Subsampling Comparisons

In [23]:
import xarray as xr
import rioxarray as rxr
import gval
from gval import CatStats
import pandas as pd
import numpy as np
import dask
import geopandas as gpd
from gval.utils.schemas import SubsamplingDf, Sample_identifiers, Subsample_identifiers
from geocube.api.core import make_geocube
import flox
from flox.xarray import xarray_reduce
from shapely.geometry import Point
from typing import Union
from gval.comparison.pairing_functions import difference
import dask as da
import time
print(time.localtime())
import pickle

time.struct_time(tm_year=2023, tm_mon=9, tm_mday=20, tm_hour=11, tm_min=22, tm_sec=3, tm_wday=2, tm_yday=263, tm_isdst=1)


## Create Subsampling DataFrame

Let's open up a geopackage with polygons to use for subsampling:

In [45]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from shapely.geometry import Polygon

In [41]:
data_path = '../data/data'
# polygons_include = gpd.read_file(f'{data_path}/subsample_continuous_polygons.gpkg')
# polygons_include.to_string()

'                                                                                                                             geometry\n0                      POLYGON ((-97.72375 29.56328, -97.72304 29.55858, -97.71574 29.55874, -97.72038 29.56409, -97.72375 29.56328))\n1  POLYGON ((-97.71604 29.55635, -97.71587 29.55196, -97.71131 29.55277, -97.71180 29.55519, -97.71267 29.55619, -97.71604 29.55635))'

In [42]:
polygons_include.to_dict()

{'geometry': {0: <POLYGON ((-97.724 29.563, -97.723 29.559, -97.716 29.559, -97.72 29.564, -9...>,
  1: <POLYGON ((-97.716 29.556, -97.716 29.552, -97.711 29.553, -97.712 29.555, -...>}}

In [59]:
polygons_continuous = gpd.GeoDataFrame({'geometry': 
                      {0: Polygon([(-97.72375, 29.56328), (-97.72304, 29.55858), (-97.71574, 29.55874), (-97.72038, 29.56409), (-97.72375, 29.56328)]),
                       1: Polygon([(-97.71604, 29.55635), (-97.71587, 29.55196), (-97.71131, 29.55277), (-97.71180, 29.55519), (-97.71267, 29.55619), (-97.71604, 29.55635)])}})
polygons_continuous.crs = 'EPSG:4326'

In [60]:
polygons_categorical = gpd.GeoDataFrame({'geometry': 
                      {0: Polygon([(-97.56735, 30.07450), (-97.51800, 29.96872), (-97.29514, 29.97237), (-97.36734, 30.04412), (-97.44813, 30.06807), (-97.56735, 30.07450)]),
                       1: Polygon([(-97.21658, 30.07011), (-97.20986, 29.95981), (-97.11101, 29.93847), (-97.12969, 30.07781), (-97.21658, 30.07011)])}})
polygons_categorical.crs = 'EPSG:4326'

In [57]:
b

,geometry
0,"POLYGON ((-97.56735 30.07450, -97.51800 29.968..."
1,"POLYGON ((-97.21658 30.07011, -97.20986 29.959..."


To use this DataFrame as a subsampling DataFrame let's use `create_subsampling_df`:

In [3]:
# polygons_include.gval.create_subsampling_df(subsampling_type=["include", "include"], inplace=True)
# polygons_include

,geometry,subsample_type,subsample_id
0,"POLYGON ((-97.72375 29.56328, -97.72304 29.558...",include,1
1,"POLYGON ((-97.71604 29.55635, -97.71587 29.551...",include,2


The DataFrame above has a geometry column, a subsample type with the value of "include" (calculating data within the geometry) or "exclude" (remove all data contained within the geometry), and subsample_id.

There is also the ability to add subsampling_weights:

In [74]:
polygons_continuous = polygons_continuous.gval.create_subsampling_df(subsampling_type=["exclude", "exclude"], subsampling_weights=[2, 1])
polygons_categorical = polygons_categorical.gval.create_subsampling_df(subsampling_type=["include", "include"])

## Continuous Compare Subsampling

In [5]:
cds = rxr.open_rasterio(f'{data_path}/candidate_continuous_1.tif', band_as_variable=True, mask_and_scale=True)
bds = rxr.open_rasterio(f'{data_path}/benchmark_continuous_1.tif', band_as_variable=True, mask_and_scale=True)

Let's use this newly created subsampling df on a continuous comparison.  For each subsample an agreement map is created and then used to calculate continuous statistics. There are four subsampling-average types:

1. <b>full-detail</b>: reports all metrics calculated on separate bands and subsamples.
2. <b>band</b>: reports all metrics on subsamples with band values averaged.
3. <b>subsample</b>: reports all metrics on bands with subsample values averaged.
4. <b>weighted</b>: reports all metrics on bands with subsample values averaged and scaled by weights.

#### Full-Detail

In [62]:
ag, met = cds.gval.continuous_compare(benchmark_map=bds,
                                      metrics=["mean_percentage_error"],
                                     subsampling_df=polygons_continuous,
                                     subsampling_average="full-detail")
met.to_dict()

{'subsample': {0: '1', 1: '1', 2: '2', 3: '2'},
 'band': {0: '1', 1: '2', 2: '1', 3: '2'},
 'mean_percentage_error': {0: 0.1259589046239853,
  1: -0.11186813563108444,
  2: 0.1671745926141739,
  3: -0.1432301551103592}}

#### Band

In [64]:
ag, met = cds.gval.continuous_compare(benchmark_map=bds,
                                      metrics=["mean_percentage_error"],
                                     subsampling_df=polygons_continuous,
                                     subsampling_average="band")
met.to_dict()

{'subsample': {0: '1', 1: '2'},
 'band': {0: 'averaged', 1: 'averaged'},
 'mean_percentage_error': {0: 0.007045384496450424, 1: 0.011972218751907349}}

#### Subsample

In [78]:
ag, met = cds.gval.continuous_compare(benchmark_map=bds,
                                      metrics=["mean_percentage_error"],
                                     subsampling_df=polygons_continuous,
                                     subsampling_average="subsample")
met.to_dict()

{'subsample': {0: 'averaged', 1: 'averaged'},
 'band': {0: '1', 1: '2'},
 'mean_percentage_error': {0: 0.1465667486190796, 1: -0.12754914537072182}}

#### Weighted

In [67]:
ag, met = cds.gval.continuous_compare(benchmark_map=bds,
                                      metrics=["mean_percentage_error"],
                                      subsampling_df=polygons_continuous,
                                      subsampling_average="weighted")
met.to_dict()

{'subsample': {0: 'averaged', 1: 'averaged'},
 'band': {0: '1', 1: '2'},
 'mean_percentage_error': {0: 0.08397260308265686, 1: -0.037289378543694816}}

## Categorical

In [53]:
# Subsampling DF
polygons_exclude = gpd.read_file(f'{data_path}/subsample_two-class_polygons.gpkg')
# polygons_exclude.gval.create_subsampling_df(subsampling_type=["exclude", "exclude"], inplace=True)

# Candidate and Benchmark
cda = rxr.open_rasterio(f'{data_path}/candidate_map_multiband_two_class_categorical.tif', mask_and_scale=True)
bda = rxr.open_rasterio(f'{data_path}/benchmark_map_multiband_two_class_categorical.tif', mask_and_scale=True)

Just as done earlier in continuous comparison, the following performs subsampling on categorical comparisons..  For each subsample an agreement map, a cross-tabulation table, and a metric table is created. There are three subsampling-average types:

1. <b>full-detail</b>: reports all metrics calculated on separate bands and subsamples.
2. <b>band</b>: reports all metrics on subsamples with band values averaged.
3. <b>subsample</b>: reports all metrics on bands with subsample values averaged.

#### Full-detail

In [75]:
ag, ctab, met = cda.gval.categorical_compare(benchmark_map=bda,
                                             metrics="all",
                                             positive_categories=[2],
                                             negative_categories=[0, 1],
                                             subsampling_df=polygons_categorical,
                                             subsampling_average="full-detail")
met.to_dict()

{'band': {0: '1', 1: '1', 2: '2', 3: '2'},
 'subsample': {0: '1', 1: '2', 2: '1', 3: '2'},
 'fn': {0: 201956.0, 1: 182394.0, 2: 68241.0, 3: 58642.0},
 'fp': {0: 761231.0, 1: 397538.0, 2: 43645.0, 3: 65969.0},
 'tn': {0: 762262.0, 1: 398349.0, 2: 1479848.0, 3: 729918.0},
 'tp': {0: 201946.0, 1: 181309.0, 2: 335661.0, 3: 305061.0},
 'accuracy': {0: 0.5002648652715194,
  1: 0.4998818547935046,
  2: 0.9419496263090856,
  3: 0.892538742141619},
 'balanced_accuracy': {0: 0.5001629939598473,
  1: 0.4995089463871429,
  2: 0.9011988338571475,
  3: 0.8779383258349349},
 'critical_success_index': {0: 0.17332441875734358,
  1: 0.23817555806899524,
  2: 0.7500016758016477,
  3: 0.7099857565771099},
 'equitable_threat_score': {0: 0.00010804127698299228,
  1: -0.0004229210476480578,
  2: 0.6960116888476541,
  3: 0.6022620032041349},
 'f_score': {0: 0.2954415948163932,
  1: 0.3847201739960745,
  2: 0.8571439515428851,
  3: 0.8303996145538584},
 'false_discovery_rate': {0: 0.790333448576949,
  1: 0.686

#### Band

In [76]:
ag, ctab, met = cda.gval.categorical_compare(benchmark_map=bda,
                                             metrics="all",
                                             positive_categories=[2],
                                             negative_categories=[0, 1],
                                             subsampling_df=polygons_categorical,
                                             subsampling_average="band")
met.to_dict()

{'subsample': {0: '1', 1: '2'},
 'band': {0: 'averaged', 1: 'averaged'},
 'fn': {0: 270197.0, 1: 241036.0},
 'fp': {0: 804876.0, 1: 463507.0},
 'tn': {0: 2242110.0, 1: 1128267.0},
 'tp': {0: 537607.0, 1: 486370.0},
 'accuracy': {0: 0.7211072457903025, 1: 0.6962102984675618},
 'balanced_accuracy': {0: 0.7006809139084974, 1: 0.6887236361110389},
 'critical_success_index': {0: 0.3333624773668676, 1: 0.4084009495236008},
 'equitable_threat_score': {0: 0.19249486119104803, 1: 0.21102574698031598},
 'f_score': {0: 0.5000327863210818, 1: 0.5799498355375926},
 'false_discovery_rate': {0: 0.599542787506434, 1: 0.48796528392623467},
 'false_negative_rate': {0: 0.33448336477660423, 1: 0.33136377758775704},
 'false_omission_rate': {0: 0.10754935603013485, 1: 0.1760282421056552},
 'false_positive_rate': {0: 0.26415480740640096, 1: 0.29118895019016516},
 'fowlkes_mallows_index': {0: 0.5162469724944239, 1: 0.5851196102503213},
 'matthews_correlation_coefficient': {0: 0.3428732020034822,
  1: 0.356124

#### Subsample

In [77]:
ag, ctab, met = cda.gval.categorical_compare(benchmark_map=bda,
                                             metrics="all",
                                             positive_categories=[2],
                                             negative_categories=[0, 1],
                                             subsampling_df=polygons_categorical,
                                             subsampling_average="subsample")
met.to_dict()

{'subsample': {0: 'averaged', 1: 'averaged'},
 'band': {0: '1', 1: '2'},
 'fn': {0: 384350.0, 1: 126883.0},
 'fp': {0: 1158769.0, 1: 109614.0},
 'tn': {0: 1160611.0, 1: 2209766.0},
 'tp': {0: 383255.0, 1: 640722.0},
 'accuracy': {0: 0.5001209918415541, 1: 0.9233890025380752},
 'balanced_accuracy': {0: 0.4998419157037689, 1: 0.8937213503088111},
 'critical_success_index': {0: 0.19895150162948627, 1: 0.7304014162939927},
 'equitable_threat_score': {0: -0.00011818049725140345, 1: 0.6575691308727142},
 'f_score': {0: 0.33187581208930095, 1: 0.8441988193216996},
 'false_discovery_rate': {0: 0.7514597697571503, 1: 0.1460865532241556},
 'false_negative_rate': {0: 0.5007132574696621, 1: 0.16529725575002768},
 'false_omission_rate': {0: 0.2487765063325223, 1: 0.054301266471772185},
 'false_positive_rate': {0: 0.4996029111228001, 1: 0.04726004363235002},
 'fowlkes_mallows_index': {0: 0.35226813927134054, 1: 0.8442534556492796},
 'matthews_correlation_coefficient': {0: -0.00027331863951123524,
  

In [14]:
print(time.localtime())

time.struct_time(tm_year=2023, tm_mon=9, tm_mday=19, tm_hour=16, tm_min=53, tm_sec=21, tm_wday=1, tm_yday=262, tm_isdst=1)
